<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para el periodo 202009. 
 

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [140]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [144]:
data1 = pd.read_csv('/content/drive/My Drive/Dataset/DBajas_ANTIGUEDAD.txt',delimiter='~')
x = data1.iloc [:,:-1].values
x = np.array(x)
y = data1.iloc [:,2].values

In [ ]:
x

In [ ]:
y

### Codificar datos categóricos

In [ ]:
'''from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder  = LabelEncoder()
labelencoder  = LabelEncoder()
x[:,1] = labelencoder.fit_transform(x[:,1])
ct = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories='auto'),[1])],remainder='passthrough')
x= np.array(ct.fit_transform(x),dtype=np.integer)'''

In [ ]:
pd.DataFrame(x)

### Evitar la trampa de variables Dummy

In [ ]:
'''x = x[:,1:]'''

### Dividimos los datos en entrenamiento y test

In [148]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.2, random_state=0)

### Escalamos los datos

In [151]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
x_train= sc_x.fit_transform(x_train)
y_train= sc_y.fit_transform(y_train.reshape(-1,1))

In [ ]:
len(x_train[:,0])

In [ ]:
pd.DataFrame(x_train)

In [ ]:
x_train

### DETERMINAR DIMENSIONES.  ELIMINACION HACIA ATRAS

In [ ]:
x_train=np.append(arr=np.ones((len(x_train[:,0]),1)).astype(int),values=x_train,axis=1)

In [ ]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8
0,1.0,-0.014876,-0.829995,-0.728744,-0.005622,2.402464,-0.026379,-0.467288,1.450313
1,1.0,-0.014876,-0.829995,1.372223,-0.005622,-0.416239,-0.026379,0.095942,-0.283268
2,1.0,-0.014876,-0.829995,1.372223,-0.005622,-0.416239,-0.026379,0.625236,-0.283268
3,1.0,-0.014876,-0.829995,-0.728744,-0.005622,2.402464,-0.026379,-0.685855,1.450313
4,1.0,-0.014876,1.204827,-0.728744,-0.005622,-0.416239,-0.026379,0.122718,-1.438989
...,...,...,...,...,...,...,...,...,...
31633,1.0,-0.014876,-0.829995,1.372223,-0.005622,-0.416239,-0.026379,1.650508,-0.861129
31634,1.0,-0.014876,-0.829995,-0.728744,-0.005622,-0.416239,-0.026379,-0.458882,-1.438989
31635,1.0,-0.014876,1.204827,-0.728744,-0.005622,-0.416239,-0.026379,-0.619538,0.872453
31636,1.0,-0.014876,-0.829995,-0.728744,-0.005622,2.402464,-0.026379,-0.782996,1.450313


In [ ]:
import statsmodels.api as sm
def elimnaAtras (x,SL):
  numvar = len(x[0])
  for i in range(0 , numvar):
    regresor_ols = sm.OLS(y, x.tolist()).fit()
    maxvar=max(regresor_ols.pvalues).astype(float)
    if maxvar>SL:
      for j in range(0,numvar-i):
        if (regresor_ols.pvalues[j].astype(float)== maxvar):
          x=np.delete(x,j,1)
  regresor_ols.summary()
  return x
SL=0.05
x_opt = x[:,[0,1,2,3,4,5,6,7]]        
x_modelo = elimnaAtras(x_opt, SL)

In [ ]:
pd.DataFrame(x_modelo)

,0,1,2,3,4
0,0,1,0,18078,6
1,0,1,0,17986,3
2,0,1,0,17952,2
3,1,0,0,95,5
4,1,0,0,369,2
...,...,...,...,...,...
39543,0,0,0,1104,6
39544,0,1,0,10271,2
39545,0,1,0,8872,5
39546,0,0,1,1790,3


### MODELO SVR

In [ ]:
from sklearn.svm import SVR
regression = SVR(kernel='rbf', C=1.0, epsilon= 0.2)
regression.fit(x_modelo[:,0:4],y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [ ]:
x_valor=sc_x.inverse_transform(muestra)
y_valor= sc_y.inverse_transform(y_muestra)
y_predic_valor= sc_y.inverse_transform(regression.predict(muestra.reshape(-1,1)))

In [ ]:
xx = []; yy = []; yy_predic=[]
ciclo= len(muestra)
for i in range(ciclo):
  xx.append([x_valor[i]])
  yy.append([y_valor[i]])
  yy_predic.append([y_predic_valor[i]])
print("x_valor = "+str(xx)+"\n")
print("y_valor = "+str(yy)+"\n")
print("y_predic"+str(yy_predic)+"\n")
# Concatenamos 
tabla = np.concatenate((xx, yy,yy_predic), axis=1)
print("Antes de organizar:"+"\n"+str(tabla)+"\n")
# Organizamos y recreamos tabla
tabla[:,0].sort()
for i in range(ciclo):
  n_orgz =  tabla[i,0]
  for ii in range(ciclo):
    n_norgz =  x_valor[ii]
    if (int(n_norgz) == int(n_orgz)):
      tabla[i,1] = y_valor[ii]
      tabla[i,2]= y_predic_valor[ii]
    else:
      pass 
print("Despues de organizar:"+"\n"+str(tabla)+"\n")

In [ ]:
# Analizando datos de test
from sklearn.preprocessing import StandardScaler
sc_x_t = StandardScaler()
sc_y_t = StandardScaler()
x_test= sc_x_t.fit_transform(x_test)
y_test= sc_y_t.fit_transform(y_test.reshape(-1,1))

In [ ]:
# MODELO SVR - Test
from sklearn.svm import SVR
regression = SVR(kernel='rbf')
regression.fit(x_test.reshape(-1,1),y_test.ravel())

In [ ]:
x_valor_t=sc_x_t.inverse_transform(x_test)
y_valor_t= sc_y_t.inverse_transform(y_test)
y_predic_valor_t= sc_y_t.inverse_transform(regression.predict(x_test.reshape(-1,1)))

In [ ]:
xx_t = []; yy_t =[]; yy_predic_t=[]
total_ciclo= len(x_test[:,0])
for i in range( total_ciclo ):
  xx_t.append(pd.DataFrame([x_valor_t[i]]))
  yy_t.append(pd.DataFrame([y_valor_t[i]]))
  yy_predic_t.append(pd.DataFrame([y_predic_valor_t[i]]))
print("x_valor_t = "+str(xx_t)+"\n")
print("y_valor_t = "+str(yy_t)+"\n")
print("y_predic_t"+str(yy_predic_t)+"\n")
# Concatenamos 
tabla_t = np.concatenate((xx_t, yy_t,yy_predic_t), axis=1)
print("Antes de organizar el test:"+"\n"+str(tabla_t)+"\n")
# Organizamos y recreamos tabla test
tabla_t[:,0].sort()
for i in range(total_ciclo) :
  n_orgz_t =  tabla_t[i,0]
  for ii in range( total_ciclo ):
    n_norgz_t =  x_valor_t[ii]
    if (int(n_norgz_t) == int(n_orgz_t)):
      tabla_t[i,1] = y_valor_t[ii]
      tabla_t[i,2]= y_predic_valor_t[ii]
    else:
      pass 
print("Despues de organizar:"+"\n"+str(tabla_t)+"\n")

In [ ]:
# Graficamos 
plt.scatter(tabla[:,0],tabla[:,1],color='Red', marker='^',alpha=0.5)
plt.plot(tabla[:,0],tabla[:,2],color='Blue', marker='^',alpha=0.5)
plt.title("Bajas vs SVR")
plt.xlabel("Periodo Baja")
plt.ylabel("Cantidad")
plt.show()

In [ ]:
# Graficamos test
plt.scatter(tabla_t[:,0],tabla_t[:,1],color='Red', marker='^',alpha=0.5)
plt.plot(tabla_t[:,0],tabla_t[:,2],color='Blue', marker='^',alpha=0.5)
plt.title("Bajas TEST vs SVR")
plt.xlabel("Periodo Baja")
plt.ylabel("Cantidad")
plt.show()

In [ ]:
'''nuevo= np.array([])
for i in range(202009 , 202012):
  nuevo= np.append(nuevo, i)
y_pred=sc_y.inverse_transform(regression.predict(sc_x.transform(nuevo.reshape(-1,1))))
pd.DataFrame(y_pred)'''

In [ ]:
# Margen de Error entrenamiento
error_muestra= []; x_train_muestra=[]
for i in range(ciclo):
  calculo_error_muestra = (( tabla[i,1] - tabla[i,2] ) / tabla[i,1] ) * 100
  calculo_error_muestra = abs(calculo_error_muestra)
  error_muestra.append(calculo_error_muestra)
  x_train_muestra.append(int(tabla[i,0]))
print("x train muestra = "+str(x_train_muestra)+"\n")
print("error_muestra = " +str(error_muestra)+"\n")
# Concatenamos  
var1=np.array([error_muestra]).T
var2=np.array([x_train_muestra]).T
tabla_error = np.concatenate((var2, var1),axis=1)
print("Antes de organizar los datos:"+str(tabla_error)+"\n")
# Organizamos y recreamos los datos
tabla_error[:,0].sort()
for i in range(ciclo) :
  n_orgz =  tabla_error[i,0]
  for ii in range( ciclo ):
    n_norgz =  var2[ii]
    if (int(n_norgz) == int(n_orgz)):
      tabla_error[i,1] = error_muestra[ii]
    else:
      pass 
print("Despues de organizar los datos:"+"\n"+str(tabla_error)+"\n")

In [ ]:
# Graficamos error muestra
plt.plot(tabla_error[:,0],tabla_error[:,1],color='Red', marker='^',alpha=0.5)
plt.title("error entrenamiento")
plt.xlabel("Periodo")
plt.ylabel("Error")
plt.show()

In [ ]:
# Margen de error Test
error_test = []; x_e_test =[]
for i in range(total_ciclo):
  calculo_error_muestra_t = (( tabla_t[i,1] - tabla_t[i,2] ) / tabla_t[i,1] ) * 100
  calculo_error_muestra_t = abs(calculo_error_muestra_t)
  error_test.append(calculo_error_muestra_t)
  x_e_test.append(int(tabla_t[i,0]))
print("x test = "+str(x_e_test)+"\n")
print("error test = " +str(error_test)+"\n")
# Concatenamos  
var1_t=np.array([error_test]).T
var2_t=np.array([x_e_test]).T
tabla_error_t = np.concatenate((var2_t, var1_t),axis=1)
print("Antes de organizar los datos:"+str(tabla_error_t)+"\n")
# Organizamos y recreamos los datos
tabla_error_t[:,0].sort()
for i in range(total_ciclo) :
  n_orgz_t =  tabla_error_t[i,0]
  for ii in range( total_ciclo ):
    n_norgz_t =  var2_t[ii]
    if (int(n_norgz_t) == int(n_orgz_t)):
      tabla_error_t[i,1] = error_test[ii]
    else:
      pass 
print("Despues de organizar los datos:"+"\n"+str(tabla_error_t)+"\n")

In [ ]:
# Graficamos error test
plt.plot(tabla_error_t[:,0],tabla_error_t[:,1],color='Blue', marker='^',alpha=0.5)
plt.title("error test")
plt.xlabel("Periodo")
plt.ylabel("Error")
plt.show()

In [ ]:
# Graficamos error 
plt.plot(tabla_error[:,0],tabla_error[:,1],color='Red', marker='^',alpha=0.5)
plt.plot(tabla_error_t[:,0],tabla_error_t[:,1],color='Blue', marker='^',alpha=0.5)
plt.title("error entrenamiento vs error test")
plt.xlabel("Periodo")
plt.ylabel("Error")
plt.show()

In [ ]:
var_x1=np.array([x_valor]).T
var_y1=np.array([y_valor]).T

In [ ]:
var_y1[:,0]

In [ ]:
var_x1_t[:,0]

In [ ]:
regression.score(var_x1[:,0], var_y1[:,0], sample_weight=None)